# T1.1: MNIST

ssh -o ProxyCommand="ssh -g -L 8889:localhost:8889 s4451856@sshgw.leidenuniv.nl -q -W U0065090:22" -g -L 8889:localhost:8889 s4451856@U0065090

In [ ]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras import optimizers
from keras import backend as K
from keras import regularizers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow.keras import Loss

import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import os

## mnist_cnn.py

In [ ]:
batch_size = 128
num_classes = 10
epochs = 50

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.xlabel("Epochs")
plt.show()

## mnist_mlp.py


In [ ]:
'''Trains a simple deep NN on the MNIST dataset.

Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

In [ ]:
batch_size = 128
num_classes = 10
epochs = 20

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.xlabel("Epochs")
plt.show()

# T1.2: Fashion MNIST

## (a) Multi-Layer Perceptron

In [ ]:
os.environ["KERAS_BACKEND"] = "tensorflow"

fashion_mnist = keras.datasets.fashion_mnist
(xTrainFull, yTrainFull), (xTest, yTest) = fashion_mnist.load_data()

xVal, xTrain = xTrainFull[:5000] / 255.0, xTrainFull[5000:] / 255.0
yVal, yTrain = yTrainFull[:5000], yTrainFull[5000:]

classNames = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
 "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]


def MLPModelTest(xTrain, yTrain, xVal, yVal, xTest, yTest,
                 learnRate=0.0001,
                 hiddenLayers=1,
                 outputActivation="softmax",
                 hiddenActivation="relu",
                 optimiser="sgd",
                 epochs=20,
                 alpha=5):

  MLPModel = Sequential()
  MLPModel.add(keras.layers.Flatten(input_shape=[28, 28]))

  for i in range(hiddenLayers):
    MLPModel.add(keras.layers.Dense(300, activation=hiddenActivation))
    
    MLPModel.add(keras.layers.Dense(100, activation=hiddenActivation))


  MLPModel.add(keras.layers.Dense(10, activation=outputActivation))

  MLPModel.compile(loss="sparse_categorical_crossentropy",
  optimizer=optimiser(learning_rate=learnRate),
  metrics=["accuracy"])

  history = MLPModel.fit(xTrain, yTrain, epochs=epochs,
                      validation_data=(xVal, yVal))

  pd.DataFrame(history.history).plot(figsize=(8, 5))
  plt.grid(True)
  plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
  plt.xlabel("Epochs")
  plt.show()

  test_loss, test_acc = MLPModel.evaluate(xTest,  yTest, verbose=2)
  print(f"Hidden layers: {hiddenLayers}")
  print(f"Optimiser: {optimiser}")
  print(f"Output Activation: {outputActivation}")
  print(f"Hidden Activation: {hiddenActivation}")
  print(f"Alpha: {alpha}")
  print(f"Epochs: {epochs}")
  print(f"Test loss: {test_loss}")
  print(f"Test accuracy: {test_acc}")

  return test_acc

In [ ]:
results = MLPModelTest(xTrain, yTrain, xVal, yVal, xTest, yTest,
                       learnRate= 0.001,
                       hiddenLayers=1,
                       optimiser=optimizers.Adam,
                       outputActivation="softplus",
                       hiddenActivation="tanh",
                       epochs=30)

print(results)

## (b) Convolutional Neural Network

In [ ]:
def CNNModelTest(xTrain, yTrain, xVal, yVal, xTest, yTest,
                 learnRate=0.0001,
                 hiddenLayers=1,
                 layerWidth=64,
                 outputActivation="softmax",
                 hiddenActivation="relu",
                 convActivation="relu",
                 optimiser=keras.optimizers.SGD,
                 epochs=50,
                 alpha=5,
                 inputShape=(28, 28, 1),
                 outputShape=10,
                 lossFunction=keras.losses.SparseCategoricalCrossentropy(from_logits=True)):

  CNNModel = Sequential()
  CNNModel.add(Conv2D(32, (3, 3), activation=convActivation, input_shape=inputShape))
  CNNModel.add(MaxPooling2D((2, 2)))
  CNNModel.add(Conv2D(64, (3, 3), activation=convActivation))
  CNNModel.add(MaxPooling2D((2, 2)))
  CNNModel.add(Conv2D(128, (3, 3), activation=convActivation))
  CNNModel.add(MaxPooling2D((2, 2)))
  CNNModel.add(Conv2D(256, (3, 3), activation=convActivation))

  CNNModel.add(Flatten())

  for i in range(hiddenLayers):
    CNNModel.add(Dense(layerWidth, activation=hiddenActivation))

  CNNModel.add(Dense(640, activation=hiddenActivation))

  CNNModel.add(Dense(outputShape, activation=outputActivation))

  CNNModel.compile(optimizer=optimiser(learning_rate=learnRate),
                loss=lossFunction,
                metrics=['accuracy'])

  #CNNModel.summary()
  history = CNNModel.fit(xTrain, yTrain, epochs=epochs,
                      validation_data=(xVal, yVal))
  
  print(history.history)

  pd.DataFrame(history.history).plot(figsize=(8, 5))
  plt.grid(True)
  plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
  plt.xlabel("Epochs")
  plt.show()

  #print(CNNModel.predict(xVal))

  val_loss, val_acc = CNNModel.evaluate(xVal,  yVal, verbose=2)
  test_loss, test_acc = CNNModel.evaluate(xTest,  yTest, verbose=2)

  print(f"Learning Rate: {learnRate}")
  print(f"Hidden layers: {hiddenLayers}")
  print(f"Layer Width: {layerWidth}")
  print(f"Optimiser: {optimiser}")
  print(f"Output Activation: {outputActivation}")
  print(f"Hidden Activation: {hiddenActivation}")
  print(f"Conv Activation: {convActivation}")
  #print(f"Alpha: {alpha}")
  print(f"Epochs: {epochs}")
  print(f"Validation loss: {val_loss}")
  print(f"Validation accuracy: {val_acc}")
  print(f"Test loss: {test_loss}")
  print(f"Test accuracy: {test_acc}")

  return test_acc, val_acc

In [ ]:
results = CNNModelTest(xTrain, yTrain, xVal, yVal, xTest, yTest,
                       learnRate=0.001,
                       hiddenLayers=2,
                       optimiser=optimizers.Adamax,
                       outputActivation="elu",
                       hiddenActivation="sigmoid",
                       convActivation="relu",
                       epochs=30)

print(results)

In [ ]:
class circularLoss(Loss):
    def call(self, y_true, y_pred):
        return tf.reduce_mean(tf.abs(tf.atan2(tf.sin(y_true - y_pred), tf.cos(y_true - y_pred))))

In [ ]:
def eval(model, xTrain, yTrain, xVal, yVal, xTest, yTest, CNN=False,
         learnRate=0.01,
        hiddenLayers=6,
        layerWidth=256,
        outputActivation="elu",
        hiddenActivation="elu",
        convActivation="relu",
        epochs=200,
        optimiser=keras.optimizers.Adagrad,
        inputShape=(75, 75, 1),
        outputShape=720,
        lossFunction=keras.losses.SparseCategoricalCrossentropy(from_logits=True)):
    
    # LearnRate
    bestLearnRate = (0.01, 0, 0)
    #for learnRate in [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]:
    #    test_acc, val_acc = model(xTrain, yTrain, xVal, yVal, xTest, yTest,
    #                              inputShape=inputShape, outputShape=outputShape,
    #                              lossFunction=lossFunction,
    #                              learnRate=learnRate)

    #    if val_acc > bestLearnRate[2]:
    #        bestLearnRate = (learnRate, test_acc, val_acc)

    #    elif val_acc < bestLearnRate[2] - 0.1:
    #        break


    #bestLoss = (circularLoss, 0, 0)
    #for loss in [circularLoss,
    #                keras.losses.MeanSquaredError,
    #                keras.losses.MeanAbsoluteError,
    #                keras.losses.MeanSquaredLogarithmicError,
    #                keras.losses.MeanAbsolutePercentageError,
    #                keras.losses.CosineSimilarity,
    #                keras.losses.Huber,
    #                keras.losses.LogCosh,
    #                keras.losses.Tversky,
    #                keras.losses.Dice]:
    #    test_acc, val_acc = model(xTrain, yTrain, xVal, yVal, xTest, yTest,
    #                              inputShape=inputShape, outputShape=outputShape,
    #                              lossFunction=loss,
    #                              learnRate=bestLearnRate[0],
    #                            epochs=100)

    #    if val_acc > bestLoss[2]:
    #        bestLoss = (loss, test_acc, val_acc)

    
    # Epochs
    bestEpochs = (125, 0, 0)
    #for epochs in range(25, 250, 25):
    #    test_acc, val_acc = model(xTrain, yTrain, xVal, yVal, xTest, yTest,
    #                              inputShape=inputShape, outputShape=outputShape,
    #                              lossFunction=lossFunction,
    #                              learnRate=learnRate,
    #                              hiddenLayers=hiddenLayers,
    #                              layerWidth=layerWidth,
    #                              outputActivation=outputActivation,
    #                              hiddenActivation=hiddenActivation,
    #                              convActivation=convActivation,
    #                              epochs=epochs,
    #                              optimiser=optimiser)

    #    if val_acc > bestEpochs[2]:
    #       bestEpochs = (epochs, test_acc, val_acc)

    #    else:
    #        break

        #elif val_acc < bestEpochs[2] - 0.1:
        #    break


    # Optimiser
    bestOptimiser = (keras.optimizers.Adagrad, 0, 0)
    #for optimiser in [keras.optimizers.SGD,
    #                keras.optimizers.RMSprop,
    #                keras.optimizers.Adam,
    #                keras.optimizers.AdamW,
    #                keras.optimizers.Adadelta,
    #                keras.optimizers.Adagrad,
    #                keras.optimizers.Adamax,
    #                keras.optimizers.Adafactor,
    #                keras.optimizers.Nadam,
    #                keras.optimizers.Ftrl,
    #                keras.optimizers.Lion]:
    #    test_acc, val_acc = model(xTrain, yTrain, xVal, yVal, xTest, yTest,
    #                              inputShape=inputShape, outputShape=outputShape,
    #                              lossFunction=lossFunction,
    #                              learnRate=learnRate,
    #                              hiddenLayers=hiddenLayers,
    #                              layerWidth=layerWidth,
    #                              outputActivation=outputActivation,
    #                              hiddenActivation=hiddenActivation,
    #                              convActivation=convActivation,
    #                              epochs=bestEpochs[0],
    #                              optimiser=optimiser)

    #    if val_acc > bestOptimiser[2]:
    #        bestOptimiser = (optimiser, test_acc, val_acc)


    # Output activation function
    bestOutActivation = ("selu", 0, 0)
    #for outputActivation in ["relu", "sigmoid", "softmax", "softplus", "softsign",
    #                "tanh", "selu", "elu", "exponential"]:
    #    test_acc, val_acc = model(xTrain, yTrain, xVal, yVal, xTest, yTest,
    #                              inputShape=inputShape, outputShape=outputShape,
    #                              lossFunction=lossFunction,
    #                              learnRate=learnRate,
    #                              hiddenLayers=hiddenLayers,
    #                              layerWidth=layerWidth,
    #                              outputActivation=outputActivation,
    #                              hiddenActivation=hiddenActivation,
    #                              convActivation=convActivation,
    #                              epochs=bestEpochs[0],
    #                              optimiser=bestOptimiser[0])

    #    if val_acc > bestOutActivation[2]:
    #        bestOutActivation = (outputActivation, test_acc, val_acc)


    # Hidden Layer activation function
    bestHiddenActivation = ("softsign", 0, 0)
    #for hiddenActivation in ["relu", "sigmoid", "softmax", "softplus", "softsign",
    #                "tanh", "selu", "elu", "exponential"]:
    #    test_acc, val_acc = model(xTrain, yTrain, xVal, yVal, xTest, yTest,
    #                              inputShape=inputShape, outputShape=outputShape,
    #                              lossFunction=lossFunction,
    #                              learnRate=learnRate,
    #                              hiddenLayers=hiddenLayers,
    #                              layerWidth=layerWidth,
    #                              outputActivation=bestOutActivation[0],
    #                              hiddenActivation=hiddenActivation,
    #                              convActivation=convActivation,
    #                              epochs=bestEpochs[0],
    #                              optimiser=bestOptimiser[0])

    #    if val_acc > bestHiddenActivation[2]:
    #        bestHiddenActivation = (hiddenActivation, test_acc, val_acc)

    
    # Convolutional layer Activation Function
    bestConvActivation = ("relu", 0, 0)
    #if CNN:
    #    for convActivation in ["relu", "sigmoid", "softmax", "softplus", "softsign",
    #                    "tanh", "selu", "elu", "exponential"]:
    #        test_acc, val_acc = CNNModelTest(xTrain, yTrain, xVal, yVal, xTest, yTest,
    #                                         inputShape=inputShape, outputShape=outputShape,
    #                                         lossFunction=lossFunction,
    #                                        learnRate=learnRate,
    #                                        hiddenLayers=hiddenLayers,
    #                                        layerWidth=layerWidth,
    #                                        outputActivation=bestOutActivation[0],
    #                                        hiddenActivation=bestHiddenActivation[0],
    #                                        convActivation=convActivation,
    #                                        epochs=bestEpochs[0],
    #                                        optimiser=bestOptimiser[0])

    #        if val_acc > bestConvActivation[2]:
    #            bestConvActivation = (convActivation, test_acc, val_acc)

    
    # Hidden Layers
    bestHidden = (6, 0, 0)
    #for hiddenLayers in range(1, 50):
    #    test_acc, val_acc = model(xTrain, yTrain, xVal, yVal, xTest, yTest,
    #                              inputShape=inputShape, outputShape=outputShape,
    #                            lossFunction=lossFunction,
    #                            learnRate=learnRate,
    #                            hiddenLayers=hiddenLayers,
    #                            layerWidth=layerWidth,
    #                            outputActivation=bestOutActivation[0],
    #                            hiddenActivation=bestHiddenActivation[0],
    #                            convActivation=bestConvActivation[0],
    #                            epochs=bestEpochs[0],
    #                            optimiser=bestOptimiser[0])

    #    if val_acc > bestHidden[2]:
    #        bestHidden = (hiddenLayers, test_acc, val_acc)

    #    elif val_acc < bestHidden[2] - 0.1:
    #        break

    
    # Hidden Layers
    bestWidth = (1, 0, 0)
    accuracyList = []
    for layerWidth in range(256, 640, 8):
        test_acc, val_acc = model(xTrain, yTrain, xVal, yVal, xTest, yTest,
                                  inputShape=inputShape, outputShape=outputShape,
                                lossFunction=lossFunction,
                                learnRate=learnRate,
                                hiddenLayers=bestHidden[0],
                                layerWidth=layerWidth,
                                outputActivation=bestOutActivation[0],
                                hiddenActivation=bestHiddenActivation[0],
                                convActivation=bestConvActivation[0],
                                epochs=bestEpochs[0],
                                optimiser=bestOptimiser[0])
    
        accuracyList.append(val_acc)

        if val_acc > bestWidth[2]:
            bestWidth = (layerWidth, test_acc, val_acc)

        elif val_acc < bestWidth[2] - 0.1:
            break


    # Regularisations
    bestAlpha = (0, 0, 0)
    #alphas = np.logspace(-10, -2, 200)
    #for alpha in alphas:
    #  test_acc, val_acc = model(xTrain, yTrain, xVal, yVal, xTest, yTest,
    #                          hiddenLayers=bestHidden[0],
    #                          outputActivation=bestOutActivation[0],
    #                          hiddenActivation=bestHiddenActivation[0],
    #                          epochs=bestEpochs[0],
    #                          optimiser=bestOptimiser[0],
    #                          alpha=alpha)

    #  if accuracy > bestAlpha[2]:
    #   bestAlpha = (alpha, accuracy)

    print(f"Best Learning Rate: {bestLearnRate[0]}")
    print(f"Best Hidden layers: {bestHidden[0]}")
    print(f"Best Width: {bestWidth[0]}")
    print(f"Best Optimiser: {bestOptimiser[0]}")
    print(f"Best Output Activation: {bestOutActivation[0]}")
    print(f"Best Hidden Activation: {bestHiddenActivation[0]}")
    print(f"Best Conv Activation: {bestConvActivation[0]}")
    print(f"Best Epochs: {bestEpochs[0]}")
    print(f"Final Val accuracy: {bestHidden[2]}")
    print(f"Final Test accuracy: {bestHidden[1]}")

    plt.figure()
    plt.plot(accuracyList, range(len(accuracyList)))
    plt.show()

    return {"bestHidden": bestHidden[0], "bestOptimiser": bestOptimiser[0],
            "bestOutActivation": bestOutActivation[0],
            "bestHiddenActivation": bestOutActivation[0],
            "bestEpochs": bestEpochs[0]}

## Experimentation and Evaluation

### MLP Model Evaluation

Naive approach, best hyperparameters:

Hidden layers: 1

Optimiser: Adam

Output Activation: SoftPlus

Hidden Activation: Tanh

Alpha:  5

Epochs: 30

Test accuracy: 0.8705999851226807

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(xTrainFull, yTrainFull), (xTest, yTest) = fashion_mnist.load_data()

xVal, xTrain = xTrainFull[:5000] / 255.0, xTrainFull[5000:] / 255.0
yVal, yTrain = yTrainFull[:5000], yTrainFull[5000:]

# MLP model eval
bestSettings = eval(MLPModelTest, xTrain, yTrain, xVal, yVal, xTest, yTest)
print(bestSettings)

### CNN Model Evaluation

Naive approach, best hyperparameters:

Hidden layers: 2

Optimiser: Adamax

Output Activation: elu

Hidden Activation: sigmoid

Conv Activation: relu

Epochs: 30

Test loss: 0.3733844459056854

Test accuracy: 0.8733000159263611

In [ ]:
# CNN model eval
bestSettings = eval(CNNModelTest, xTrain, yTrain, xVal, yVal, xTest, yTest, CNN=True)
print(bestSettings)

### CIFAR-10

In [ ]:
cifar = keras.datasets.cifar10
(xTrainFull, yTrainFull), (xTest, yTest) = cifar.load_data()

xVal, xTrain = xTrainFull[:5000] / 255.0, xTrainFull[5000:] / 255.0
yVal, yTrain = yTrainFull[:5000], yTrainFull[5000:]

In [ ]:
#Hidden layers: 2
#Optimiser: Adamax
#Output Activation: elu
#Hidden Activation: sigmoid
#Conv Activation: relu
#Epochs: 30
#Test loss: 0.3733844459056854
#Test accuracy: 0.8733000159263611

test_acc, val_acc = CNNModelTest(xTrain, yTrain, xVal, yVal, xTest, yTest,
                            learnRate=0.001,
                          hiddenLayers=2,
                          outputActivation="elu",
                          hiddenActivation="sigmoid",
                          convActivation="relu",
                          epochs=30,
                          optimiser=keras.optimizers.Adamax,
                          inputShape=(32, 32, 3))

print(test_acc, val_acc)

# T2.1 Tell-the-time Network

## (a) Classification

24 Classes

In [ ]:
xSmallClock = np.load("data/75/images.npy")
ySmallClock = np.load("data/75/labels.npy")

ySmallClock = np.array([int(time[0] + (time[1] // 30)) for time in ySmallClock])  

xSmallClock = np.reshape(xSmallClock, (-1, 75, 75, 1))
xSmallClock = xSmallClock / 255.0

xTrain, xTest, yTrain, yTest = train_test_split(xSmallClock,
                                                ySmallClock,
                                                test_size=0.2)
xVal, xTest, yVal, yTest = train_test_split(xTest, yTest, test_size=0.5)

test_acc, val_acc = CNNModelTest(xTrain, yTrain, xVal, yVal, xTest, yTest,
                                 learnRate=0.001,
                                hiddenLayers=2,
                                outputActivation="softmax",
                                hiddenActivation="relu",
                                convActivation="relu",
                                epochs=30,
                                optimiser=keras.optimizers.Adam,
                                inputShape=(75, 75, 1),
                                outputShape=24)

print(test_acc, val_acc)

720 Classes

In [ ]:
xSmallClock = np.load("data/75/images.npy")
ySmallClock = np.load("data/75/labels.npy")

ySmallClock = np.array([int((time[0] * 60) + time[1]) for time in ySmallClock])  

xSmallClock = np.reshape(xSmallClock, (-1, 75, 75, 1))
xSmallClock = xSmallClock / 255.0

xTrain, xTest, yTrain, yTest = train_test_split(xSmallClock,
                                                ySmallClock,
                                                test_size=0.2)
xVal, xTest, yVal, yTest = train_test_split(xTest, yTest, test_size=0.5)

#test_acc, val_acc = CNNModelTest(xTrain, yTrain, xVal, yVal, xTest, yTest,
#                          learnRate=0.01,
#                          hiddenLayers=6,
#                          layerWidth=256,
#                          outputActivation="elu",
#                          hiddenActivation="elu",
#                          convActivation="relu",
#                          epochs=200,
#                          optimiser=keras.optimizers.Adagrad,
#                          inputShape=(75, 75, 1),
#                          outputShape=720)

#print(test_acc, val_acc)

bestSettings = eval(CNNModelTest, xTrain, yTrain, xVal, yVal, xTest, yTest,
                          learnRate=0.01,
                          hiddenLayers=6,
                          layerWidth=512,
                          outputActivation="elu",
                          hiddenActivation="elu",
                          convActivation="relu",
                          epochs=200,
                          optimiser=keras.optimizers.Adagrad,
                          inputShape=(75, 75, 1),
                          outputShape=720,
                          lossFunction=keras.losses.SparseCategoricalCrossentropy(from_logits=True))

print(bestSettings)

## (b) Regression

In [ ]:
xSmallClock = np.load("data/75/images.npy")
ySmallClock = np.load("data/75/labels.npy")

ySmallClock = np.array([float(time[0] + (time[1] / 60)) for time in ySmallClock])

xSmallClock = np.reshape(xSmallClock, (-1, 75, 75, 1))
xSmallClock = xSmallClock / 255.0
    
xTrain, xTest, yTrain, yTest = train_test_split(xSmallClock,
                                                ySmallClock,
                                                test_size=0.2)
xVal, xTest, yVal, yTest = train_test_split(xTest, yTest, test_size=0.5)

test_acc, val_acc = CNNModelTest(xTrain, yTrain, xVal, yVal, xTest, yTest,
                                 learnRate=0.01,
                                hiddenLayers=50,
                                layerWidth=8,
                                outputActivation="softmax",
                                hiddenActivation="relu",
                                convActivation="relu",
                                epochs=200,
                                optimiser=keras.optimizers.Adam,
                                inputShape=(75, 75, 1),
                                outputShape=1,
                                lossFunction=keras.losses.mean_absolute_error)

print(test_acc, val_acc)

#bestSettings = eval(CNNModelTest, xTrain, yTrain, xVal, yVal, xTest, yTest, CNN=True,
#                    inputShape=(75, 75, 1), outputShape=1, lossFunction=circularLoss)
#print(bestSettings)